### Cointegration x Multiple Linear Regression -> Markov Chain 

Feed multiple linear regression output into the transitional matrix for the Markov Chain transition probabilities between states.

In [31]:
# Libaries 
# Libraries and Modules used
import MetaTrader5 as mt5 
import pandas as pd
from sklearn.linear_model import LinearRegression
import ta
import warnings
warnings.filterwarnings("ignore")
import numpy as np
# warnings.filterwarnings("ignore")
mt5.initialize()
# Replace following with your MT5 Account Login
account=51127988 # 
password="Aar2frM7"
server = 'ICMarkets-Demo'

# Function to get rates from MT5 
def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1['close']

### Cointegration Code

### Multiple Linear Regression 

### Markov Chain

In [133]:
class MarkovChain:
    
    # Adjust Accordingly 
    def __init__(self, price_data):
        self.data = pd.DataFrame(price_data)
        self.states = ['s_bull', 'w_bull', 'w_bear', 's_bear']
    
    def compute_returns(self):
        self.data['returns'] = self.data['close'].pct_change()
        
    # Temporary states, to amend to more sophisticated/specific states
    def classify_state(self, ret):
        if ret > 0.01:
            return 's_bull'
        elif ret > 0.003 and ret < 0.01:
            return 'w_bull'
        elif ret < 0.002 and ret > -0.002:
            return 'doji'
        elif ret < -0.003 and ret > -0.01:
            return 'w_bear'
        else:
            return 's_bear'
        
    def classify_states(self):
        self.data['state'] = self.data['returns'].apply(self.classify_state)

    # def transition_matrix(self):
    

In [134]:
data = get_rates("EURUSD.a", mt5.TIMEFRAME_D1, 500)
data

0      1.16066
1      1.15793
2      1.16127
3      1.15545
4      1.15675
        ...   
495    1.05719
496    1.05031
497    1.05661
498    1.05732
499    1.05700
Name: close, Length: 500, dtype: float64

In [135]:
a = MarkovChain(data)
a.compute_returns()
a.classify_states()
a.data

,close,returns,state
0,1.16066,NaN,s_bear
1,1.15793,-0.002352,s_bear
2,1.16127,0.002884,s_bear
3,1.15545,-0.005012,w_bear
4,1.15675,0.001125,doji
...,...,...,...
495,1.05719,-0.001907,doji
496,1.05031,-0.006508,w_bear
497,1.05661,0.005998,w_bull
498,1.05732,0.000672,doji
